# Article Generation with Reflection
## Introduction

Building on what we learned in Lab 5, we'll now apply the Reflection pattern to create a article generation system. As a quick recap, the Reflection pattern involves iteratively refining an LLM's output through self-critique or external review.

A Reflection pattern can be depicted in the following diagram:

<img src="../imgs/reflection-pattern.png" width="600">

In this lab, we'll implement the Reflection pattern using Bedrock Agents. Let's start by building our Article Generation agent!


## Article Generation agent


First, we are going to create an Article Generation agent. This agent is responsible for creating drafts of high-quality news, sports, and entertainment articles.

Next, we will create an Article Reviewer agent. This agent is responsible for performing a detailed review of a given article draft generated by the Article Generation agent.

The following depicts the architecture diagram for this module using Bedrock Agents.

<img src="../imgs/lab7-architecture-diagram-article-writer-agent.png" width="800"> 

## Create an Article Generation agent using Amazon Bedrock Agents
In this section we declare global variables that will be act as helpers during entire notebook and you will start to create your first agent.


In [ ]:
import boto3
import time


sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.amazon.nova-pro-v1:0'
]

## Importing helper functions
In following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, knowledge bases for bedrock creation, and accessing data on S3.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

## Creating an agent

Create the Article Generation agent

The role of this agent is to create an Article based on provided input, and potential feedback from the Reviewer agent.

We will provide the following instructions to our agent:

In [ ]:
agent_description = """You are an expert article writer creating engaging news, sports, and entertainment content tailored to audiences and publication requirements."""
agent_instruction = """## Agent Role
You are an expert journalist who transforms raw news facts and related research material into compelling, professionally written news articles. Your articles must be accurate, engaging, and adhere to high journalistic standards.

## Input Processing
1. Carefully analyze all news facts provided about the event
2. Review all contextual research about entities mentioned in the news facts
3. Identify key information, connections, and the most newsworthy elements
4. Organize information in order of importance (inverted pyramid style)

## Article Creation Guidelines
1. Create a concise, attention-grabbing headline that accurately represents the story
2. Write approximately 800 words (adjust if specifically requested otherwise)
3. Begin with a strong lead paragraph that answers the key questions (who, what, when, where, why, how)
4. Structure the article with the most important information first, followed by supporting details
5. Incorporate relevant context from the research materials where appropriate
6. Use direct quotes from sources when available
7. Maintain a neutral, objective tone throughout
8. Ensure factual accuracy - only use information provided in the input materials
9. Avoid speculation, personal opinions, or unsupported claims
10. Use concise, clear language accessible to general readers
11. Break up text with appropriate paragraphs for readability
12. Include a conclusion that ties the story together or points to future developments

## Feedback Integration
1. When receiving feedback from review agents, analyze it thoroughly
2. Make all requested changes that align with journalistic standards
3. Revise for clarity, accuracy, balance, or completeness as directed
4. If feedback contains contradictory requests, prioritize factual accuracy and journalistic ethics
5. Return the revised article with all improvements implemented

## Output Format
The output should consist of only:
1. A headline
2. The article body

Do not include:
- Tags like "Headline:" or "Article:"
- Explanations about your writing process
- Notes about sources or research
- Additional formatting markers
- Thoughts or reflections on the article
- Metadata or structural elements

## Example Output Structure:
Major Discovery Transforms Scientific Understanding

Scientists at Stanford University have announced a breakthrough discovery that challenges existing theories...

[Article continues for approximately 800 words]"""

In [ ]:
article_generation_agent_name = "article_generation_agent"
article_generation_agent = agents.create_agent(
    article_generation_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False
)

time.sleep(20)
article_generation_agent

Update the agent to increase the max ouput tokens 

In [ ]:
agents.update_agent_max_tokens(agents.get_agent_id_by_name(article_generation_agent_name),max_tokens=4096)

Prepare the agent so that it can be invoked.

In [ ]:
agents.prepare(article_generation_agent_name)

## Testing the agent
At this point, you have successfully created an Article Generation agent with given instructions.
Next, we'll test the agent independently to verify that the agent follows the directions and works as intended. 
To do so, we will use the default alias created: `TSTALIASID` which allows you to invoke a draft version of your agent:

In [ ]:
# This is a sample output of research returned by the research agent
research_prompt = """
## Researched Entities

- **entity_id: 1**
  **text:** NeuraHealth Solutions
  **type:** ORGANIZATION
  **subtype:** Corporation
  **confidence:** 0.95
  **research:**
    - NeuraHealth Solutions is a leading healthcare technology company specializing in AI-powered diagnostic and healthcare decision support systems.
    - The company's flagship product, MediScan, uses machine learning algorithms to detect and diagnose medical conditions with high accuracy.
    - NeuraHealth Solutions reported a revenue of $870 million in 2023, marking a 23% growth from the previous year.
    - The company has expanded its partnership with Cleveland Clinic and acquired DataMed for $85 million in 2023.
    - NeuraHealth Solutions has a market capitalization of $12.8 billion and is listed on NASDAQ under the ticker symbol NRHL.

- **entity_id: 2**
  **text:** MediScan
  **type:** PRODUCT
  **subtype:** Medical Diagnostic Platform
  **confidence:** 0.95
  **research:**
    - MediScan is an AI-powered diagnostic platform developed by NeuraHealth Solutions.
    - The platform demonstrated 94% accuracy in early disease detection across a trial of 12,000 patients.
    - MediScan is expected to receive FDA approval by the third quarter of 2024.
    - The platform will be piloted at five major hospital networks starting next month.
    - Initial focus areas include cardiovascular disease, diabetes, and early cancer detection.

- **entity_id: 3**
  **text:** Dr. Eliza Chen
  **type:** PERSON
  **subtype:** Expert
  **confidence:** 0.95
  **research:**
    - Dr. Eliza Chen is the Chief Medical Officer at NeuraHealth Solutions.
    - She revealed that the MediScan system was trained on 50 million anonymized patient records.
    - Dr. Chen has a background in medical diagnostics and has contributed to several research papers in the field.

- **entity_id: 4**
  **text:** Marcus Williams
  **type:** PERSON
  **subtype:** Executive
  **confidence:** 0.95
  **research:**
    - Marcus Williams is the Chief Executive Officer of NeuraHealth Solutions.
    - He announced the launch of MediScan at the annual developer conference.
    - Under his leadership, the company invested $450 million in research and development over three years.

- **entity_id: 5**
  **text:** Raj Patel
  **type:** PERSON
  **subtype:** Executive
  **confidence:** 0.95
  **research:**
    - Raj Patel is the Senior Vice President of Product Development at NeuraHealth Solutions.
    - He confirmed that FDA approval for MediScan is expected by the third quarter.
    - Raj Patel has been with the company since 2016 and has played a key role in the development of several products.


## New Facts

- NeuraHealth Solutions announced its new medical diagnostic platform called "MediScan" at their annual developer conference yesterday.
- The system demonstrated 94% accuracy in early disease detection across a trial of 12,000 patients.
- Dr. Eliza Chen, Chief Medical Officer at NeuraHealth, revealed the system was trained on 50 million anonymized patient records.
- NeuraHealth CEO Marcus Williams stated the company invested $450 million in research and development over three years.
- The platform will be piloted at five major hospital networks starting next month.
- Senior Vice President of Product Development, Raj Patel, confirmed that FDA approval is expected by the third quarter.
- Initial focus areas include cardiovascular disease, diabetes, and early cancer detection."""
print(research_prompt)

In [ ]:
%%time
draft_article = agents.invoke(research_prompt, 
    article_generation_agent[0], enable_trace=True,
    trace_level = "all"
)


Here's the output:

In [ ]:
print(draft_article)

## Create an Alias
We've just completed a test query against the Article Generation agent using the default alias `TSTALIASID`.
The default Alias is a quick way to test an agent before integrating it into your application.
When creating a multi-agent collaboration, it's required to create an Alias explicitly so that it can be used by other agents. This is to ensure the agent is tested and validated the functionality as expected.
Since we've tested and validated our agent, let's now create an Alias for it:

In [ ]:
article_generation_alias_id, article_generation_agent_alias_arn = agents.create_agent_alias(
    article_generation_agent[0], 'v1'
)

## Saving information
Let's store the environment variables to be used in other labs.

In [ ]:
article_generation_agent_arn = agents.get_agent_arn_by_name(article_generation_agent_name)
article_generation_agent_id = article_generation_agent[0]

%store article_generation_agent_arn
%store article_generation_agent_alias_arn
%store article_generation_alias_id
%store article_generation_agent_name
%store article_generation_agent_id
%store draft_article

# Next step
We have just created an Article Generation agent responsible for creating a draft version of the article. In the next notebook, we'll build an agent that review and provide feedback for the draft version of the article to improve the quality of the output.